In [2]:
import pandas as pd
import numpy as np


In [4]:
data_df=pd.read_csv('data_top_users_1.csv')

In [2]:
from tqdm import tqdm


In [3]:
chunks=pd.read_csv('user-interactions.csv', chunksize=1000000)

In [4]:
#Read in csv in chunks


data_df=[]
for chunk in tqdm(chunks):
    data_df.append(chunk)
data_df=pd.concat(data_df)

121it [05:40,  2.81s/it]


In [6]:
#Grouping by user id and aggragating on read_percent count to know how many datapoints correspond with a user reading a book and sorting in descending order

user_counts=data_df.groupby('user_id')['read_percent'].count().sort_values(ascending=False)

In [8]:
#Creating mask for a million datapoints
boolean_mask=np.cumsum(user_counts)<1000000

In [9]:
top_users=user_counts[boolean_mask]


In [10]:
data_top=data_df[data_df['user_id'].isin(top_users.index)]

In [12]:
data_top

,user_id,pratilipi_id,read_percent,updated_at
269,5506791953988402,1377786227805897,100.0,2022-03-23 00:06:48.927
289,5506791953988402,1377786227836741,92.0,2022-03-23 00:06:44.165
357,5506791955913619,1377786222441036,17.0,2022-03-23 00:06:27.603
684,5506791953988402,1377786227773852,100.0,2022-03-23 00:04:59.652
770,5506791953906904,1377786223237230,100.0,2022-03-23 00:04:32.069
...,...,...,...,...
120404870,5506791961037165,1377786226466869,100.0,2022-01-21 00:01:57.359
120404917,5506791961037165,1377786224983554,100.0,2022-01-21 00:01:45.545
120405036,5506791961037165,1377786227311277,100.0,2022-01-21 00:01:17.408
120405133,5506791949627388,1377786221901035,13.0,2022-01-21 00:00:56.871


In [18]:
data_top['read_percent'].value_counts()

100.000000    858301
99.000000      13549
50.000000       6274
10.000000       5935
25.000000       5142
               ...  
6.666667           1
81.818182          1
36.363636          1
7.142857           1
5.555556           1
Name: read_percent, Length: 136, dtype: int64

In [13]:
#dropping duplicates to ensure only one instance of (user_id,pratilip_id) occurs to prevent inconsistency
data_top.drop_duplicates(['user_id','pratilipi_id'],keep='first',inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10200\2926370219.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top.drop_duplicates(['user_id','pratilipi_id'],keep='first',inplace=True)


In [15]:

data_top['updated_at']=pd.to_datetime(data_top['updated_at'])

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10200\141040514.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top['updated_at']=pd.to_datetime(data_top['updated_at'])


In [16]:
data_top.sort_values('updated_at',inplace=True)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10200\352041338.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_top.sort_values('updated_at',inplace=True)


In [17]:
data_top.to_csv('data_top_users_1.csv')

In [1]:
data_df=pd.read_csv('data_top_users_1.csv')

NameError: name 'pd' is not defined

In [3]:
import surprise
from surprise import Reader, Dataset

In [4]:
reader = Reader(rating_scale=(0, 100))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(data_df[['user_id', 'pratilipi_id', 'read_percent']], reader)

NameError: name 'data_df' is not defined

In [ ]:
dataset=data.build_full_trainset()

In [ ]:
from surprise import SVD #evaluate

##TODO use other alogorthms like non-negative matrix factorization
#from surprise import NMF

# svd
algo = SVD()

##TODO cross validation
#evaluate(algo, data, measures=['RMSE'])



In [ ]:
fit_dataset=algo.fit(dataset)

In [ ]:
## Run Inference
recos={}
for uid in tqdm(data_df['user_id'].unique()):
    not_read=(data_df['pratilipi_id'][data_df['user_id']!=uid]).unique()
    preds=[]
    
    for story in not_read:
        prediction=fit_dataset.predict(uid,story)
        preds.append(prediction)
    recos[uid]=preds

In [ ]:
## output top 5 predictions sorted by estimated reading completion
out={uid:sorted(reco,key=lambda x:x[3])[:5] for uid,reco in recos.items()}
out={uid:[rec[1] for rec in reco] for uid,reco in out.items()}

In [ ]:
pd.DataFrame(out).transpose().to_csv('pratilipi_top_recommendations.csv')